# Creating Ensemble Model

VotingRegressor pretrained modelleri desteklemediği için terkardan kendimiz eğitmek zorundayız.

## Setup Enviroment

In [1]:
import joblib
import re
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV,cross_validate

In [50]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import make_scorer

In [3]:
hgbr_model = joblib.load("hgbr.pkl")

In [4]:
lgbmr_model = joblib.load("lgbm_model.pkl")

## Dataset Process...

### Loading Datasets

In [5]:
df2_ = pd.read_excel("kaggle_test2.xlsx",index_col="Unnamed: 0")
df_ = pd.read_csv("dataset.csv", encoding="unicode-escape", sep=",",on_bad_lines="skip",index_col="Unnamed: 0")

In [6]:
df = df_.copy()

### Preprocess

In [7]:
# Değişken isimlerini küçültüyorum
df.columns = [col.lower() for col in df.columns]

# Bağımlı değişkenin türevlerini overfit olmaması için çıkartıyorum.
df.drop(["md","ml","mw","ms","mb"],axis=1,inplace=True)

# Date değişkeninin oluşturulması
df["olus tarihi"] = df["olus tarihi"].astype(str)
df["olus zamani"] = df["olus zamani"].astype(str)
# tip değişkeni 2 sınıf ve SM sınıfından sadece 2 gözlem var. O yüzden dropluyorum bu kolonu.
df["depremin_olus_zamani"] = df["olus tarihi"] + " " + df["olus zamani"]
df.drop(["olus tarihi", "olus zamani", "tip"], axis = 1, inplace = True)
df["depremin_olus_zamani"] = pd.to_datetime(df["depremin_olus_zamani"])

# Çoğullanmış ID'leri düşürüyorum.
df.drop_duplicates(subset=["deprem kodu"],inplace = True)

# Türkçe karakterleri İngilizce karakterlere dönüştüren bir çeviri sözlüğü oluşturalım
turkce_ingilizce_ceviri = {
    'ç': 'c',
    'ğ': 'g',
    'ı': 'i',
    'ö': 'o',
    'ş': 's',
    'ü': 'u',
    'Ç': 'C',
    'Ğ': 'G',
    'İ': 'I',
    'Ö': 'O',
    'Ş': 'S',
    'Ü': 'U'
}
df["yer"] = df["yer"].apply(lambda yer: yer.translate(str.maketrans(turkce_ingilizce_ceviri)))

# Şehir isimlerini çekmek için düzenli ifade kullanalım

def extract_cities(dataframe,column):
  sadece_sehirler = []
  for yer in dataframe[column]:
      eslesme = re.search(r'\((.*?)\)', yer)  # Parantez içindekileri kontrol et
      if eslesme:
          sadece_sehirler.append(eslesme.group(1))  # Parantez içindeki değeri ekle
      else:
          sadece_sehirler.append(yer)  # Parantez içinde değer yoksa doğrudan ekle
  return sadece_sehirler

df["temizlenmis_yer"] = extract_cities(df,"yer")

# Köşeli parantezleri atıyoruz. Böylelikle o şehir/alandaki depremlere odaklanmış oluyoruz.
df["temizlenmis_yer"] = df["temizlenmis_yer"].apply(lambda yer: re.sub(r'\[.*?\]', '', yer))

# Yerin neresi olduğu belli olmadığı için çıkartıyorum.
df = df[~(df["temizlenmis_yer"] == "#NAME?")]

In [8]:
df.temizlenmis_yer.nunique()

368

### Joining Two Tables

In [9]:
df = pd.concat([df,df2_],axis=0,ignore_index=True)
df.head()

,no,deprem kodu,enlem,boylam,derinlik,xm,yer,depremin_olus_zamani,temizlenmis_yer
0,1.0,2.023043e+13,38.3392,37.7633,8.7,4.3,KEPEZ-AKCADAG (MALATYA) [East 1.7 km],2023-04-30 13:01:27.690,MALATYA
1,2.0,2.023043e+13,40.8077,31.0708,3.7,3.5,PASAKONAGI- (DUZCE) [South West 0.9 km],2023-04-30 04:02:53.490,DUZCE
2,3.0,2.023043e+13,35.6422,34.0252,22.4,3.5,AKDENIZ,2023-04-30 02:51:22.150,AKDENIZ
3,4.0,2.023043e+13,37.8623,36.2185,5.0,4.0,KARAKUYU-SAIMBEYLI (ADANA) [East 1.4 km],2023-04-29 19:48:32.870,ADANA
4,5.0,2.023043e+13,38.0220,36.4457,5.0,3.7,YIRICEK-GOKSUN (KAHRAMANMARAS) [North West 2....,2023-04-29 17:58:29.200,KAHRAMANMARAS


In [10]:
df.temizlenmis_yer.nunique()

368

## Feature Engineering

### Creating DateTime

In [11]:
def creating_datetime_feature(dataframe):
    dataframe["ay"] = dataframe["depremin_olus_zamani"].dt.month
    dataframe["yil"] = dataframe["depremin_olus_zamani"].dt.year
    dataframe["ay_gun"] = dataframe["depremin_olus_zamani"].dt.day
    dataframe["hafta_gun"] = dataframe["depremin_olus_zamani"].dt.dayofweek
    dataframe["saat"] =  dataframe["depremin_olus_zamani"].dt.hour
    dataframe["aksam"] = ((dataframe['saat'] >= 18) & (dataframe['saat'] <= 23)) | ((dataframe['saat'] >= 0) & (dataframe['saat'] <= 4))
    dataframe['yıl_gün'] = dataframe["depremin_olus_zamani"].dt.dayofyear
    dataframe['yıl_hafta'] = dataframe["depremin_olus_zamani"].dt.weekofyear
    dataframe["ay_basi"] = dataframe["depremin_olus_zamani"].dt.is_month_start.astype(int)
    dataframe["ay_sonu"] = dataframe["depremin_olus_zamani"].dt.is_month_end.astype(int)
    dataframe["çeyreklik"] = dataframe["depremin_olus_zamani"].dt.quarter
    return dataframe
df = creating_datetime_feature(df)

C:\Users\oftomurcuk\AppData\Local\Temp\ipykernel_3980\4216593417.py:9: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  dataframe['yıl_hafta'] = dataframe["depremin_olus_zamani"].dt.weekofyear


In [12]:
# Akşam değişkeni Integer tipine dönüştürülüyor. 
df["aksam"] = df['aksam'].astype(int)

### Lag Feature

In [13]:
# random noise. Bağımlı değişkenden türeteceğimiz featurelarda, overfitin önüne geçmek için gürültü oluşturacağız.
def random_noise(dataframe):
    return np.random.normal(scale=1.6, size=(len(dataframe)))

In [14]:
# Zamana göre sıralıyoruz Zaman Serilerini.
df = df.sort_values(by = ["depremin_olus_zamani","temizlenmis_yer"],
               axis = 0)

In [15]:
# lag features. Shift ile gecikmeyi sağlıyoruz. 
# Random noise ile gürültü ekliyoruz. 
# Transform ile de zaten dönüştürme yapıyoruz.
def lag_features(dataframe, lags):
    for lag in lags:
        dataframe['deprem_lag_' + str(lag)] = dataframe.groupby(["temizlenmis_yer"])['xm'].transform(
            lambda x: x.shift(lag)) + random_noise(dataframe)
    return dataframe

In [16]:
# çeşitli shiftler/lagler/gecikmeler girelim. Gecikme  o zamanki değer demekti unutma. 3 ay ve katları olacak şekilde bakıyorum! Quarter aldım.
# Gelecekten Not: Şuanlık veri setinde gün gün gözlem var. Bunları çeyreklik yapınca SMAPE değerimiz arttı!
df = lag_features(df, [1, 3, 7, 30, 60, 90, 180, 360])

In [17]:
df.isnull().sum()

no                       2208
deprem kodu              2208
enlem                       0
boylam                      0
derinlik                    0
xm                       2208
yer                      2208
depremin_olus_zamani        0
temizlenmis_yer             0
ay                          0
yil                         0
ay_gun                      0
hafta_gun                   0
saat                        0
aksam                       0
yıl_gün                     0
yıl_hafta                   0
ay_basi                     0
ay_sonu                     0
çeyreklik                   0
deprem_lag_1             2208
deprem_lag_3             2208
deprem_lag_7             2576
deprem_lag_30            4999
deprem_lag_60            6880
deprem_lag_90            8363
deprem_lag_180          11382
deprem_lag_360          14531
dtype: int64

### Rolling Mean Features

In [18]:
# Rolling Mean Feature
# Window parametresi, adım sayısını belirtmek için kullanılıyor.
def roll_mean_features(dataframe, windows):
    for window in windows:
        dataframe['deprem_roll_mean_' + str(window)] = dataframe.groupby(["temizlenmis_yer"])['xm']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) + random_noise(
            dataframe)
    return dataframe

In [19]:
# Min Periodu 10 yaptığımız için min window'a da 10 girebiliyoruz.
df = roll_mean_features(df, [10, 30, 60, 90, 180, 360])

In [20]:
df.isnull().sum()

no                       2208
deprem kodu              2208
enlem                       0
boylam                      0
derinlik                    0
xm                       2208
yer                      2208
depremin_olus_zamani        0
temizlenmis_yer             0
ay                          0
yil                         0
ay_gun                      0
hafta_gun                   0
saat                        0
aksam                       0
yıl_gün                     0
yıl_hafta                   0
ay_basi                     0
ay_sonu                     0
çeyreklik                   0
deprem_lag_1             2208
deprem_lag_3             2208
deprem_lag_7             2576
deprem_lag_30            4999
deprem_lag_60            6880
deprem_lag_90            8363
deprem_lag_180          11382
deprem_lag_360          14531
deprem_roll_mean_10      3741
deprem_roll_mean_30      3226
deprem_roll_mean_60      3226
deprem_roll_mean_90      3226
deprem_roll_mean_180     3226
deprem_rol

### Exponential Weighted Mean Features

In [21]:
# Shift ile gecikme veriyor
# ewm ile üs alıyor,
# en sonrada ortalamasını alıp transform ile dönüştürüyoruz.
def ewm_features(dataframe, alphas, lags):
    for alpha in alphas:
        for lag in lags:
            dataframe['deprem_ewm_alpha_' + str(alpha).replace(".", "") + "_lag_" + str(lag)] = \
                dataframe.groupby(["temizlenmis_yer"])['xm'].transform(lambda x: x.shift(lag).ewm(alpha=alpha).mean())
    return dataframe

In [22]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [1, 3, 7, 30, 60, 90, 180, 360]

In [23]:
df = ewm_features(df, alphas, lags)

In [24]:
df.isnull().sum()

no                              2208
deprem kodu                     2208
enlem                              0
boylam                             0
derinlik                           0
                               ...  
deprem_ewm_alpha_05_lag_30      4999
deprem_ewm_alpha_05_lag_60      6880
deprem_ewm_alpha_05_lag_90      8363
deprem_ewm_alpha_05_lag_180    11382
deprem_ewm_alpha_05_lag_360    14531
Length: 74, dtype: int64

### One Hot Encoding

In [25]:
df = pd.get_dummies(df, columns=['temizlenmis_yer'])

In [26]:
df.shape

(22111, 441)

### Log1p Dönüşümü

In [27]:
df['xm'] = np.log1p(df["xm"].values)

### Time_based Feature and Validation Sets

In [28]:
# 2023 3 ayı validasyon, öncekilerde eğitim

# 2023'nin başına kadar (2022'nın sonuna kadar) train seti.
train = df.loc[(df["depremin_olus_zamani"] < "2023-01-01"), :]

# 2023'nin ilk 3'ayı validasyon seti.
val = df.loc[(df["depremin_olus_zamani"] >= "2023-01-01") & (df["depremin_olus_zamani"] < "2023-04-01"), :]

In [29]:
# train edilirken işimize yaramayacak olan kolonları çıkartalım. Buraya daha sonra enlem ve boylam eklenecek
cols = [col for col in df.columns if col not in ['deprem kodu',"no","yer","depremin_olus_zamani","temizlenmis_yer","xm"]]

In [30]:
# train => X_train, Y_train ....... val => X_val, Y_val
Y_train = train['xm']
X_train = train[cols]

Y_val = val['xm']
X_val = val[cols]

In [31]:
Y_train.shape, X_train.shape, Y_val.shape, X_val.shape

((18424,), (18424, 436), (1383,), (1383, 436))

## Model Developing

**Burada tekrardan Optuna kullanmıyorum.** Çünkü daha önce kullandım ve sonuçları test ettim. O yüzden eski parametreleri tekrardan kullanarak modeli eğitiyorum.

### HistGradientBoositngRegressor

In [32]:
optimized_params = {
    'max_iter': 774,
    'max_depth': 2,
    'min_samples_leaf': 18,
    'learning_rate': 0.0018862398822892638
}

In [33]:
hgbr_model = HistGradientBoostingRegressor(
    max_iter=optimized_params['max_iter'],
    max_depth=optimized_params['max_depth'],
    min_samples_leaf=optimized_params['min_samples_leaf'],
    learning_rate=optimized_params['learning_rate'],
    random_state=42  # You can set the random seed for reproducibility
)


In [35]:
hgbr_model.fit(X_train, Y_train)

# Make predictions
y_pred = hgbr_model.predict(X_val)

In [36]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

def hist_gradient_boosting_smape(preds, train_data):
    labels = train_data
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

In [37]:
hist_gradient_boosting_smape(y_pred, Y_val)

('SMAPE', 8.231142277443645, False)

### LightGBMRegressor

In [41]:
def smape(preds, target):
    n = len(preds)
    masked_arr = ~((preds == 0) & (target == 0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds - target)
    denom = np.abs(preds) + np.abs(target)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val


def lgbm_smape(preds, train_data):
    labels = train_data.get_label()
    smape_val = smape(np.expm1(preds), np.expm1(labels))
    return 'SMAPE', smape_val, False

In [81]:
lgbmr_model_parameters = {'boosting_type': 'gbdt',
             'learning_rate': 0.19986850305793402,
             'subsample': 0.9778880064060288,
             'subsample_freq': 5,
             'feature_fraction': 0.6153367760626559,
             'min_child_samples': 17,
             'min_data_in_leaf': 67,
             'objective': None,
             'num_iterations': 30000}

In [82]:
lgbm_model = LGBMRegressor(**lgbmr_model_parameters)
lgbm_model.fit(X_train, Y_train)

C:\Users\oftomurcuk\AppData\Roaming\Python\Python311\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] feature_fraction is set=0.6153367760626559, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6153367760626559
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] feature_fraction is set=0.6153367760626559, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6153367760626559
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004708 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15124
[LightGBM] [Info] Number of data points in the train set: 18424, number of used features: 115
[LightGBM] [Info] Start training from score 1.600840


LGBMRegressor(feature_fraction=0.6153367760626559,
              learning_rate=0.19986850305793402, min_child_samples=17,
              min_data_in_leaf=67, num_iterations=30000,
              subsample=0.9778880064060288, subsample_freq=5)

In [83]:
y_pred_val = lgbm_model.predict(X_val,
                                #num_iteration=model.best_iteration
                                )

smape(np.expm1(y_pred_val), np.expm1(Y_val))

[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] feature_fraction is set=0.6153367760626559, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6153367760626559


9.02614816819916

In [45]:
"""
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)

lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

lgbmr_model = lgb.train(lgbmr_model_parameters, lgbtrain,
                  valid_sets=[lgbtrain, lgbval],
                  num_boost_round =30000,
                  # early_stopping_rounds=lgb_params['early_stopping_rounds'],
                  feval=lgbm_smape,
                  # verbose_eval=1000
                  )
"""

C:\Users\oftomurcuk\AppData\Roaming\Python\Python311\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005394 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15124
[LightGBM] [Info] Number of data points in the train set: 18424, number of used features: 115
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 1.600840


In [46]:
y_pred_val = lgbmr_model.predict(X_val,
                                #num_iteration=model.best_iteration
                                )

smape(np.expm1(y_pred_val), np.expm1(Y_val))

9.02614816819916

## Creating VotingRegressor 

**VotingRegression pretrained model kullanmıyor.** Bu yüzden modeli eğitmemiz gerekiyor. Fakat alternatif var:

### Creating SMAPE for VotingRegressor 

In [54]:
def smape_scorer(y_true, y_pred):
    n = len(y_true)
    masked_arr = ~((y_pred == 0) & (y_true == 0))
    y_pred, y_true = y_pred[masked_arr], y_true[masked_arr]
    num = np.abs(y_pred - y_true)
    denom = np.abs(y_pred) + np.abs(y_true)
    smape_val = (200 * np.sum(num / denom)) / n
    return smape_val

### Creating VotingRegressor

In [48]:
voting_regressor = VotingRegressor(estimators=[
    ('lgbmr_model', lgbmr_model),
    ('hgbr_model', hgbr_model),
])

In [56]:
smape_scorer_make = make_scorer(smape_scorer, greater_is_better=False)

In [58]:
scores = -cross_val_score(voting_regressor, X_train, Y_train, cv=5, scoring=smape_scorer_make,n_jobs=-1, error_score="raise")

ValueError: The estimator Booster should be a regressor.

In [61]:
def voting_regressor_smape(models, X, y):
    # model listesi
    regressors = [(f"model_{idx}", model) for idx, model in enumerate(models)]
    
    # VotingRegressor oluşturuyorum
    voting_regressor = VotingRegressor(estimators=regressors)
    
    # Use SMAPE as the scoring metric during cross-validation
    smape_scorer_make = make_scorer(smape_scorer, greater_is_better=False)
    scores = -cross_val_score(voting_regressor, X_train, Y_train, cv=5, scoring=smape_scorer_make,n_jobs=-1, error_score="raise")

    # Calculate the mean SMAPE score across cross-validation folds
    mean_smape_score = np.mean(scores)
    return mean_smape_score


In [84]:
voting_regressor_smape((hgbr_model,lgbm_model),X_train,Y_train)

4.069787514538628

In [87]:
regressors = [(f"model_{idx}", model) for idx, model in enumerate((hgbr_model,lgbm_model))]
voting_regressor = VotingRegressor(estimators=regressors)

In [89]:
voting_regressor.fit(X_train,Y_train)

C:\Users\oftomurcuk\AppData\Roaming\Python\Python311\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] feature_fraction is set=0.6153367760626559, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6153367760626559
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] min_data_in_leaf is set=67, min_child_samples=17 will be ignored. Current value: min_data_in_leaf=67
[LightGBM] [Warning] feature_fraction is set=0.6153367760626559, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6153367760626559
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005033 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15124
[LightGBM] [Info] Number of data points in the train set: 18424, number of used features: 115
[LightGBM] [Info] Start training from score 1.600840
[LightGBM] [Warning] 

array([1.59206108, 1.58392806, 1.56711518, ..., 1.60512348, 1.56349074,
       1.58462308])

**Aşırı Önemli Not:** <br>
Lighgbm'in kendi API'si Sklearn gibi model olarak dönmüyor. Fakat Sklearn'de dönüyor. Bu <br>
yüzden kendi API'ını kullanmak yerin Sklearn kulladnım. Bu yüzden Microsoft ürünleri beş para etmez.

### Alternative VotingRegression For Pretrained Models

VotingRegression'un Zat-ı Ali'leri pretrained kabul etmediği için şu yoluda deneyebiliriz:

In [85]:

# preds1 = model1.predict(X_test)
# preds2 = model2.predict(X_test)

# ensemble_predictions = (preds1 + preds2) / 2  

# mse = mean_squared_error(y_test, ensemble_predictions)
# print(f"Mean Squared Error (MSE) for the ensemble: {mse}")

## Joblib

In [95]:
joblib.dump(voting_regressor,"votingw_regressor.pkl")

['votingw_regressor.pkl']